[20 卷积层里的填充和步幅【动手学深度学习v2】](https://www.bilibili.com/video/BV1Th411U7UN?spm_id_from=333.999.0.0)

6.3.1 填充
- 如上所述，在应⽤多层卷积时，我们常常丢失边缘像素。由于我们通常使⽤小卷积核，因此对于任何单个卷积，我们可能只会丢失⼏个像素。但随着我们应⽤许多连续卷积层，累积丢失的像素数就多了。
- 解决这个问题的简单⽅法即为填充（padding）：在输⼊图像的边界填充元素（通常填充元素是0 ）。
- 例如，在图6.3.1 中，
- 我们将3X3 输⼊填充到5X5，那么它的输出就增加为4X4。阴影部分是第⼀个输出元素以及⽤于输出计算的输⼊和核张量元素：0X0 + 0X1 + 0X2 + 0X3 = 0。

### 6.3.2 步幅
- 在计算互相关时，卷积窗口从输⼊张量的左上⻆开始，向下和向右滑动。在前⾯的例⼦中，我们默认每次滑动⼀个元素。但是，有时候为了⾼效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。
- 我们将每次滑动元素的数量称为步幅（stride）。到⽬前为⽌，我们只使⽤过⾼度或宽度为1 的步幅，那么如何使⽤较⼤的步幅呢？图6.3.2 是垂直步幅为3，⽔平步幅为2 的⼆维互相关运算。着⾊部分是输出元素以及⽤于输出计算的输⼊和内核张量元素：0 X 0 + 0 X 1 + 1 X 2 + 2 X 3 = 8、0 X 0 + 6 X 1 + 0 X 2 + 0 X 3 = 6。
- 可以看到，为了计算输出中第⼀列的第⼆个元素和第⼀⾏的第⼆个元素，卷积窗口分别向下滑动三⾏和向右滑动两列。但是，当卷积窗口继续向右滑动两列时，没有输出，因为输⼊元素⽆法填充窗口（除⾮我们添加另⼀列填充）。

In [14]:
import torch
from torch import nn

In [15]:
def comp_conv2d(conv2d,X):
    X = X.reshape((1,1)+X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

In [16]:
# padding为填充
# 这里kernel_size=3相当于直接声明一个3*3卷积核
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
# 创建一个随机的8*8矩阵
X = torch.rand(size=(8,8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [17]:
# 对不同的高度和宽度机进行填充
# stride=2就是步幅参数
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [18]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape
# 为什么是2*2 详情见P233

torch.Size([2, 2])

### Q/A
- 填充一般来说让填充变成正方形比较好
- 一般来说步幅选择为1，计算量过大的时候可以考虑增大步幅
- 卷积核边长一般选奇数，效果其实差不多，但有助于使矩阵更规范
- 多层3*3卷积核和大核少层效果相当，但小核计算快
- 目前业内通用做法是第一层使用较大的卷积核，后面依旧采用3*3小核进行卷积